In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Notebook to summarise uses of ExternalGalaxyAstrometry package

In [ ]:
import ExternalGalaxyAstrometry as EGA

Input file is (by default) drawn from the Gaia archive. In this example we draw it from a 8 degree radius around the photometric centre of the LMC

For simplicity while this software is testig this is saved into a local file

In [ ]:
input_name ='testfile.h5'

dataLMC = pd.read_hdf(input_name,'table')